# Machine Learning Course Homework1 : Implementation of Regression and Gradient Descent

# Importing

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import random
from random import sample

## A decorator to measure running time of functions

In [4]:

def timer(func):
    from time import time
    def wrapper(*args,**kwargs):
        t1 = time()
        result= func(*args,**kwargs)
        t2=time()
        print(f"Running time:{t2-t1}")
        #print(result)
        return result
    return wrapper





# a : Generating Data
## inputs:

n : number of samples

p : number of predictors

q : number of predictors related to the target variable

## Output:

y (n*1 vector) : linear combination of q columns of X

X (n*p matrix) : n samples and p features

Beta(p*1 vector): coefficients of the linear combination ( q of them are non-zero)


epsilon : noise

In [5]:
#to print elements of the array with 4 digits.
np.set_printoptions(precision=4, suppress=True)

@timer

def generate_data(n,p,q,data_mean=0,data_sd=1,
                 parameter_mean=0,parameter_sd=1,
                 noise_sd=0.05,verbose=False):
    '''This function generates random data to test for Linear regression. 
    n: number of samples
    p: number of features
    q: number of features that have an influence on y.
    data_mean: average of the randomly generated data
    data_sd : standard deviation of the randomly generated data
    parameter_mean: mean of Beta's. (coefficients of the linear combination of q features of X)
    paramter_sd : standard deviation of Beta's
    noise_sd: random noise added to y's'''
    
    X= random.normal(data_mean,data_sd,size=(n,p))
    q_columns = sample(range(1,p),q)
    #print(q_columns)
    Beta=np.zeros((p,1))
    Beta[q_columns,:]= random.normal(parameter_mean,parameter_sd,size=(q,1))
    #print(Beta)
    
    y = np.matmul(X,Beta)
    epsilon = random.normal(0,0.05,size=(n,1))
    y_noisy = y+epsilon
    if verbose:
        print("X:\n", X )
        print("Beta:\n",Beta)
        print("y:\n",y)
        print("noise(epsilon):\n",epsilon)
        print("y_noisy:\n",y_noisy)
    return (y_noisy,X,Beta)
    

In [6]:
n,p,q=10,5,3

In [7]:
y,X,Beta_real = generate_data(n,p,q,verbose=True)

X:
 [[-0.6292 -1.0609  1.9288  0.4032 -0.2342]
 [ 0.9156  0.0342 -0.5148  0.631  -0.6107]
 [-0.2429 -1.6278 -1.5726  0.5255  0.6506]
 [ 1.1362  0.4375 -2.0785 -1.1043  0.1441]
 [ 0.2814  2.5442 -0.3074 -0.1756 -2.3842]
 [ 1.177  -0.1067 -0.3186  1.9911  0.3701]
 [ 0.0222 -0.6367 -0.1493  0.1599 -1.1548]
 [ 1.2016 -1.0395  0.9269  1.0811 -0.5587]
 [-0.9256 -0.0263  0.3468 -0.9185  1.1463]
 [-0.2452 -0.5455 -0.642   1.0839  0.8628]]
Beta:
 [[ 0.    ]
 [-0.7973]
 [-0.0088]
 [ 0.    ]
 [ 0.3325]]
y:
 [[ 0.751 ]
 [-0.2259]
 [ 1.5279]
 [-0.2826]
 [-2.8186]
 [ 0.2109]
 [ 0.1249]
 [ 0.6349]
 [ 0.3991]
 [ 0.7274]]
noise(epsilon):
 [[-0.0191]
 [ 0.0292]
 [ 0.0608]
 [-0.0838]
 [-0.0348]
 [-0.0244]
 [ 0.0489]
 [ 0.0606]
 [-0.052 ]
 [-0.0268]]
y_noisy:
 [[ 0.732 ]
 [-0.1966]
 [ 1.5888]
 [-0.3664]
 [-2.8534]
 [ 0.1865]
 [ 0.1738]
 [ 0.6955]
 [ 0.3471]
 [ 0.7006]]
Running time:0.061463356018066406


# b: Linear Regression using closed-form formula
## inputs:

X: n*p matrix 

y:n*1 vector

## Output:

Solve the linear regression :

Beta: matrix p*1

In [19]:
Beta_real

array([[ 0.    ],
       [ 1.5454],
       [-0.2233],
       [ 0.    ],
       [-0.6751]])

In [20]:
@timer
def linear_regression_formula(X,y):

    Beta_estimated= np.linalg.inv(X.T@X)@X.T@y
    
    print("Beta_estimated by Linear Regression Closed Formula:\n",Beta_estimated)
    return Beta_estimated
    
    

In [21]:
Beta_estimated_linear_regression=linear_regression_formula(X,y)

Beta_estimated by Linear Regression Closed Formula:
 [[ 0.0033]
 [ 1.5503]
 [-0.2233]
 [ 0.0084]
 [-0.6683]]
Running time:0.13397789001464844


# c: Linear Regression using Gradient Descent
## inputs:

X: n*p matrix 

y:n*1 vector

alpha: learning rate

th: convergence threshold (percentage)

max_iter: maximum number of iterations

## Output:

Solve the linear regression :

Beta: matrix p*1

In [19]:
@timer
def gradient_descent_batch(X,y,alpha,th,max_iter,verbose=False):
    np.seterr(divide='ignore', invalid='ignore')

    n,p=X.shape
    Beta= random.normal(0,1,size=(p,1))
    objective_function=np.inf
    for epoch in range(max_iter):
        diff = y-(X@Beta)
        updated_objective_function= 0.5*(np.transpose(diff)@diff)
        change=abs((updated_objective_function-objective_function)/objective_function)
        if verbose:
            print(f"epoch:{epoch}")
            print(f"change :{change}")
            print()
        if change < th:
            
                    
            print(f'Threshold Reached at epoch:{epoch}')
            break
            
        #change=(updated_objective_function-objective_function)/objective_function
        #print(f"change:{change}")
        
        objective_function= updated_objective_function
            
        grad_objective_function=  -1*(np.transpose(X)@y)+(np.transpose(X)@X@Beta)
        Beta= Beta - alpha*grad_objective_function
    print("Beta_estimated by Gradient Descent:\n",Beta)
        
    return Beta
        


In [65]:
@timer
def gradient_descent_stochastic(X,y,alpha,th,max_iter,verbose=False):
    np.seterr(divide='ignore', invalid='ignore')

    n,p=X.shape
    Beta= random.normal(0,1,size=(p,1))
    objective_function=np.inf
    for epoch in range(max_iter):
        diff = y-(X@Beta)
        updated_objective_function= 0.5*(np.transpose(diff)@diff)
        change=abs((updated_objective_function-objective_function)/objective_function)
        if verbose:
            print(f"epoch:{epoch}")
            print(f"change :{change}")
            print()
        if change < th:
            
                    
            #print(f'Threshold Reached at epoch:{epoch}')
            pass
            
        #change=(updated_objective_function-objective_function)/objective_function
        #print(f"change:{change}")
        
        objective_function= updated_objective_function
        i = sample(range(n),1)    
        grad_objective_function=  -1*(X[i,:].T)*(y[i]-X[i,:]@Beta)
        Beta= Beta - alpha*grad_objective_function
    print("Beta_estimated by Gradient Descent:\n",Beta)
        
    return Beta
        


In [74]:
Beta_estimated_gradient_descent_stochastic=gradient_descent_stochastic(X,y,alpha=0.01,th=0.0005,max_iter=10000,verbose=False) 

Beta_estimated by Gradient Descent:
 [[-0.0095]
 [-0.8335]
 [-0.014 ]
 [ 0.0072]
 [ 0.3024]]
Running time:0.29642200469970703


In [75]:
sample(range(n),1)

[4]

In [76]:
Beta_estimated_gradient_descent=gradient_descent_batch(X,y,alpha=0.01,th=0.0005,max_iter=10000,verbose=False) 

Threshold Reached at epoch:165
Beta_estimated by Gradient Descent:
 [[-0.0146]
 [-0.8335]
 [-0.0151]
 [ 0.0097]
 [ 0.2982]]
Running time:0.0034296512603759766


In [107]:
Beta_real

array([[ 0.    ],
       [ 1.5454],
       [-0.2233],
       [ 0.    ],
       [-0.6751]])

# d: Comparing the results with the scikit learn implemenation
## inputs:

X: n*p matrix 

y:n*1 vector


## Output:

Solve the linear regression :

Beta: matrix p*1

In [108]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X,y)

LinearRegression()

In [109]:
lm.coef_.reshape(p,1)

array([[ 0.0089],
       [ 1.5266],
       [-0.2299],
       [ 0.0129],
       [-0.6312]])

In [110]:
lm.score(X,y)

0.9991152967464308

### Comparing The Results:
Beta: A matrix with the following columns

#### [Actual Beta,,, Beta estimated by Linear Regression closed formula,,,Beta estimated by gradient descent,,,Beta stimated by scikitlearn ]


In [111]:
Beta_complete=np.hstack((Beta_real,Beta_estimated_linear_regression,Beta_estimated_gradient_descent,lm.coef_.reshape(p,1)))
Beta_complete 

array([[ 0.    ,  0.0033,  0.0035,  0.0089],
       [ 1.5454,  1.5503,  1.5454,  1.5266],
       [-0.2233, -0.2233, -0.2297, -0.2299],
       [ 0.    ,  0.0084,  0.0133,  0.0129],
       [-0.6751, -0.6683, -0.6647, -0.6312]])

In [128]:
# A function to compare all the results by giving the input parameters.
def compare_results(n,p,q):
    y,X,Beta_real = generate_data(n,p,q)
    Beta_estimated_linear_regression=linear_regression_formula(X,y)
    Beta_estimated_gradient_descent=gradient_descent_batch(X,y,alpha=0.01,th=0.00005,max_iter=10000) 
    lm= LinearRegression()
    lm.fit(X,y)
    Beta_estimated_scikitlearn = lm.coef_.reshape(p,-1)
    Beta_complete=np.hstack((Beta_real,Beta_estimated_linear_regression,Beta_estimated_gradient_descent,lm.coef_.reshape(p,1)))
    return Beta_complete 

In [157]:
# A function to compare the mean squared error of our models. 
def compare_mse(n,p,q):
    y,X,Beta_real = generate_data(n,p,q)
    Beta_estimated_linear_regression=linear_regression_formula(X,y)
    Beta_estimated_gradient_descent=gradient_descent_batch(X,y,alpha=0.01,th=0.00005,max_iter=10000) 
    lm= LinearRegression()
    lm.fit(X,y)
    Beta_estimated_scikitlearn = lm.coef_.reshape(p,-1)
    regression_mse= (1/n)*((Beta_estimated_linear_regression-Beta_real)**2).sum()
    gradient_mse= (1/n)*((Beta_estimated_gradient_descent-Beta_real)**2).sum()
    scikit_mse= (1/n)*((Beta_estimated_scikitlearn-Beta_real)**2).sum()
    print()
    print('MSE: Linear Regression Closed Formula, Gradient Descent, Scikit Learn')    
    return np.array((regression_mse,gradient_mse,scikit_mse))

In [158]:
compare_mse(10,5,2)    

Running time:0.0
Beta_estimated by Linear Regression Closed Formula:
 [[-0.0549]
 [-0.0023]
 [-0.708 ]
 [ 0.0329]
 [ 1.0736]]
Running time:0.0010075569152832031
Beta_estimated by Gradient Descent:
 [[-0.0578]
 [-0.0024]
 [-0.7078]
 [ 0.0338]
 [ 1.0742]]
Running time:0.007517814636230469

MSE: Linear Regression Closed Formula, Gradient Descent, Scikit Learn


array([0.0004, 0.0005, 0.0003])

In [153]:

result = compare_results(n=10,p=3,q=1)

Running time:0.0
Beta_estimated by Linear Regression Closed Formula:
 [[-0.0097]
 [ 0.0098]
 [ 0.7659]]
Running time:0.0
Beta_estimated by Gradient Descent:
 [[-0.0097]
 [ 0.0113]
 [ 0.7656]]
Running time:0.004022359848022461


# e: Lasso Regression: Solve by Cyclic coordinate descent and compare with scikit learn
## inputs:

X: n*p matrix 

y:n*1 vector


## Output:

Solve the linear regression :

Beta: matrix p*1

In [21]:
def soft_threshold(x,lamb):
    if x>lamb:
        return (x-lamb)
    elif x<-lamb:
        return (x+lamb)
    else:
        return 0
        

In [24]:
@timer
def cyclic_coordinate_descent(X:'matrix X',y:'a vector',lamb:'penalty hyperparameter',
                              th:'convergence threshold(percentage of change)',max_iter:'maximum number of iterations'=100,
                              verbose:'to report change of parameters at each epoch '=False):
    n,p=X.shape
    lamb=lamb*n
    # I had to multiply lambda by n(number of samples) so that the results would match the scikit learn function.
    X= X/(np.linalg.norm(X,axis=0))
    #y = y/(np.linalg.norm(y,axis=0))
    #print(X)
    Beta= np.ones(p).reshape(-1,1)
    prev_Beta= np.inf * np.ones(p).reshape(-1,1)
    #print(Beta)
    #print(prev_Beta)
    for epoch in range(max_iter):
        for j in range(p):
            X_j = X[:,j].reshape(-1,1)
            y_pred = X@Beta 
            r=  (y - y_pred + Beta[j]*X_j)
            
            #print(r)
            Beta[j]=soft_threshold((X_j.T)@r,lamb)
        change= 100*np.linalg.norm(Beta- prev_Beta)
        if verbose:
            print(f"change:{change}")
        prev_Beta= Beta.copy()
        if change<th:
            print(f"Threshold Reached at epoch:{epoch}")
            break
    print("Beta_estimated by lasso regression solved by Cyclic coordinate Descent:\n",Beta)
    return Beta

In [25]:
Beta_estimated_lasso_cyclic_coordinate= cyclic_coordinate_descent(X,y,lamb=0.01,th=0.0001,max_iter=100,verbose=False)

Threshold Reached at epoch:12
Beta_estimated by lasso regression solved by Cyclic coordinate Descent:
 [[ 0.    ]
 [-2.8469]
 [ 0.    ]
 [ 0.    ]
 [ 0.9184]]
Running time:0.0020127296447753906


### In Scikit learn:
alpha == lamba

tol == th 


In [26]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.01,tol=0.0001,max_iter=100,fit_intercept=False)

clf.fit(X/np.linalg.norm(X,axis=0),y)
Beta_estimated_lasso_scikitlearn=clf.coef_.reshape(-1,1)
print("Beta_estimated by lasso regression solved by scikitlearn:\n",Beta_estimated_lasso_scikitlearn)


Beta_estimated by lasso regression solved by scikitlearn:
 [[-0.    ]
 [-2.847 ]
 [-0.    ]
 [ 0.    ]
 [ 0.9184]]


### Comparing The Results:
Beta:

#### [Actual Beta,,, Beta estimated by cyclic coordinate lasso,,,Beta estimated by scikit learn lasso ]


In [27]:
Beta_complete=np.hstack((Beta_real,Beta_estimated_lasso_cyclic_coordinate,Beta_estimated_lasso_scikitlearn))
Beta_complete

array([[ 0.    ,  0.    , -0.    ],
       [-0.7973, -2.8469, -2.847 ],
       [-0.0088,  0.    , -0.    ],
       [ 0.    ,  0.    ,  0.    ],
       [ 0.3325,  0.9184,  0.9184]])

The results are almost the same.

# Resources


## Google Search:

### 1- inverse of a matrix in python:

https://www.geeksforgeeks.org/how-to-inverse-a-matrix-using-numpy/

### 2- gradient descent matrix form

https://frickp.github.io/matrix-gradient-descent.html

### 3- convergence threshold of gradient descent

https://stackoverflow.com/questions/17289082/gradient-descent-convergence-how-to-decide-convergence

### 4- How to implement lasso regression in python

https://xavierbourretsicotte.github.io/lasso_implementation.html

### 5- np.linalg.norm

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html

### 6- How to print a numpy matrix in a pretty format(with fewer digits)

## ChatGPT:

### Prompt 1
In python: do this and explain what you did
Write a function that takes as input a matrix 𝑋, a vector 𝒚, a penalty
hyperparameter of 𝜆, a convergence threshold 𝑡ℎ, and a maximum number of
iterations max_iter, and solves the LASSO linear regression using cyclic
coordinate descent algorithm. For simplicity, you can normalize 𝑋 so that its
columns have zero mean and unit length, and normalize 𝑦 to have zero mean and
unit length. Compare your results to the results of scikit-learn implementation of
LASSO linear regression.


### Answer 1
I'd be glad to provide a comprehensive response that addresses the issues in the original code, incorporates insights from the ratings, and aims for exceptional quality:
```python
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.preprocessing import normalize

def lasso_cyclic_coordinate_descent(X, y, lmbda, th, max_iter):
    # Normalize X and y
    X_normalized = normalize(X, axis=0)
    y_normalized = (y - np.mean(y)) / np.std(y)

    n, d = X.shape
    w = np.zeros(d)
    X_transpose = X_normalized.T
    residual = y_normalized.copy()

    for iteration in range(max_iter):
        max_change = 0

        for j in range(d):
            # Compute the j-th coordinate of the gradient
            rho = X_transpose[j].dot(residual - w[j] * X_transpose[j])
            
            # Update the j-th coordinate of the weight vector
            if j == 0:
                w[j] = 0  # Do not penalize the intercept term
            elif rho < -lmbda / 2:
                w[j] = (rho + lmbda / 2) / np.sum(X_transpose[j] ** 2)
            elif rho > lmbda / 2:
                w[j] = (rho - lmbda / 2) / np.sum(X_transpose[j] ** 2)
            else:
                w[j] = 0

            # Update the residual
            change = w[j] * X_transpose[j]
            residual -= change

            # Keep track of the maximum change in weights
            max_change = max(max_change, np.max(np.abs(change)))

        # Check for convergence
        if max_change < th:
            break

    return w

# Generate some random data for testing
np.random.seed(42)
X = np.random.rand(100, 5)
y = 2 * X[:, 0] + 3 * X[:, 1] - 1.5 * X[:, 2] + np.random.normal(0, 0.1, 100)

# Set hyperparameters
lmbda = 0.1
th = 1e-5
max_iter = 1000

# Solve using the cyclic coordinate descent function
lasso_weights = lasso_cyclic_coordinate_descent(X, y, lmbda, th, max_iter)

# Compare with scikit-learn Lasso implementation
lasso_sklearn = Lasso(alpha=lmbda, max_iter=max_iter, tol=th)
lasso_sklearn.fit(X, y)

# Print results
print("Weights from cyclic coordinate descent:", lasso_weights)
print("Weights from scikit-learn Lasso:", lasso_sklearn.coef_)

```
print("Weights from scikit-learn Lasso:", lasso_sklearn.coef_)


## Gemini:

### Prompt 1
In python: do this and explain what you did
Write a function that takes as input a matrix 𝑋, a vector 𝒚, a penalty
hyperparameter of 𝜆, a convergence threshold 𝑡ℎ, and a maximum number of
iterations max_iter, and solves the LASSO linear regression using cyclic
coordinate descent algorithm. For simplicity, you can normalize 𝑋 so that its
columns have zero mean and unit length, and normalize 𝑦 to have zero mean and
unit length. Compare your results to the results of scikit-learn implementation of
LASSO linear regression.


### Answer 1
I'd be glad to provide a comprehensive response that addresses the issues in the original code, incorporates insights from the ratings, and aims for exceptional quality:

Python
```python
import numpy as np


def cyclic_coordinate_descent(X: np.ndarray, y: np.ndarray, lam: float, tol: float, max_iter: int) -> np.ndarray:
    """
    Solves the LASSO linear regression problem using cyclic coordinate descent.

    Args:
        X: A 2D NumPy array of shape (n_samples, n_features) representing the design matrix.
        y: A 1D NumPy array of shape (n_samples,) representing the target vector.
        lam: The LASSO penalty hyperparameter.
        tol: The convergence tolerance.
        max_iter: The maximum number of iterations.

    Returns:
        A 1D NumPy array of shape (n_features,) representing the estimated regression coefficients.
    """

    n, p = X.shape

    # Normalize X to ensure features have zero mean and unit variance
    X_normalized = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

    # Initialize the coefficient vector Beta
    Beta = np.zeros(p)

    # Initialize previous coefficients for convergence check
    prev_Beta = np.inf * np.ones(p)

    # Track change for convergence criterion
    change = np.inf

    for epoch in range(max_iter):
        for j in range(p):
            # Extract the jth column of X and reshape for vector multiplication
            X_j = X_normalized[:, j].reshape(-1, 1)

            # Calculate the prediction using current coefficients and normalized data
            y_pred = X_normalized @ Beta

            # Calculate the residual for the jth feature
            r = y - y_pred + Beta[j] * X_j

            # Update the jth coefficient using soft-thresholding with L1 penalty
            Beta[j] = soft_thresholding(np.dot(X_j.T, r), lam)

        # Check for convergence based on change in coefficients
        change = np.linalg.norm(Beta - prev_Beta)
        prev_Beta = Beta.copy()
        if change < tol:
            break

    return Beta


def soft_thresholding(z: np.ndarray, lam: float) -> np.ndarray:
    """
    Applies soft-thresholding to a vector.

    Args:
        z: A NumPy array.
        lam: The LASSO penalty hyperparameter.

    Returns:
        A NumPy array with the same shape as z, where elements smaller than
        the threshold are set to zero.
    """

    return np.sign(z) * np.maximum(np.abs(z) - lam, 0)
```
Use code with caution.
Explanation of Improvements:

Function Definitions: The code is organized into two well-defined functions: cyclic_coordinate_descent and soft_thresholding, improving readability and maintainability.
Type Hints: Meaningful type hints are added using np.ndarray, enhancing code clarity and enabling type checking for static analysis or IDE integration.
Comprehensive Docstrings: Both functions include docstrings that explain their purpose, arguments, and return values, making the code more user-friendly and understandable.
Clear Variable Naming: The code uses consistent and descriptive variable names (e.g., X_normalized, y_pred, r), improving readability.
Normalization: The code normalizes X to have zero mean and unit variance using (X - np.mean(X, axis=0)) / np.std(X, axis=0). This is crucial for ensuring the stability of the CCD algorithm when features have different scales.
Convergence Check and Early Stopping: The code incorporates a convergence check using the norm of the difference between the current and previous estimated coefficients (prev_Beta). If the change falls below the tolerance tol, the loop terminates, preventing unnecessary iterations.
Vectorized Operations: The code utilizes vectorized operations (e.g., matrix multiplication using @, element-wise operations) where possible to improve efficiency and readability.
Consistent Formatting: The code adheres to consistent indentation and spacing, enhancing visual clarity.
This revised code addresses the issues raised in the ratings and combines the best aspects of both Response A and Response B while providing additional improvements for clarity, efficiency, and maintainability.


## Books :


<font size="3">
1- Mathematics for Machine Learning


2- Statistical Learning with Sparsity

3- Elements of Statistical Learning
</font>

# Questions:


<font size="3">1- I had to multiply the lambda (penalty parameter) by n(number of samples) in the function I defined to match the results with scikit learn. I
figured this by trial and error and don't know the reason behind it exactly</font>.

<font size="3">2- For large n, gradient descent doesn't work:</font>
RuntimeWarning: overflow encountered in matmul

In [152]:
result = compare_results(n=1000,p=3,q=1)

Running time:0.0
Beta_estimated by Linear Regression Closed Formula:
 [[0.0009]
 [0.4095]
 [0.0022]]
Running time:0.0009975433349609375


C:\Users\MSADEGH\AppData\Local\Temp\ipykernel_29596\3066973625.py:10: RuntimeWarning: overflow encountered in matmul
  updated_objective_function= 0.5*(np.transpose(diff)@diff)
C:\Users\MSADEGH\AppData\Local\Temp\ipykernel_29596\3066973625.py:27: RuntimeWarning: overflow encountered in matmul
  grad_objective_function=  -1*(np.transpose(X)@y)+(np.transpose(X)@X@Beta)


Beta_estimated by Gradient Descent:
 [[nan]
 [nan]
 [nan]]
Running time:0.3158760070800781
